In [251]:
import os
import numpy as np
import pandas as pd

In [252]:

file_path_1 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/compustat_profit.csv' # Fundamental data
df = pd.read_csv(file_path_1)
df['datadate'] = pd.to_datetime(df['datadate'])
df['rdq'] = pd.to_datetime(df['rdq'])
df = df.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr','costat'])

df = df.sort_values(by=['tic', 'rdq'])

df.head()

,gvkey,datadate,tic,ajexq,rdq,actq,cshoq,epsfiq,teqq,cshfdy,oancfy,dvpspq
1243,1690,1981-06-30,AAPL,224.0,1981-07-22,201.262,55.136,0.21,NaN,NaN,NaN,0.0
1244,1690,1981-09-30,AAPL,224.0,1981-10-20,227.104,55.309,0.19,NaN,NaN,NaN,0.0
1245,1690,1981-12-31,AAPL,224.0,1982-01-22,249.160,55.421,0.24,NaN,NaN,NaN,0.0
1246,1690,1982-03-31,AAPL,224.0,1982-04-15,263.140,56.488,0.24,NaN,NaN,NaN,0.0
1247,1690,1982-06-30,AAPL,224.0,1982-07-16,283.437,56.743,0.26,NaN,NaN,NaN,0.0


In [253]:
file_path_2 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/prices_profit.csv' # price data
df2 = pd.read_csv(file_path_2)
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2.drop(columns=['PERMNO'])

df2 = df2.sort_values(by=['TICKER', 'date'])
df2 = df2.rename(columns={'TICKER': 'tic'})

df2['PRC'] = df2['PRC'].abs()
df2['OPENPRC'] = df2['OPENPRC'].abs()


df2.head()

,date,tic,PRC,OPENPRC
312596,1980-12-12,AAPL,28.8125,NaN
312597,1980-12-15,AAPL,27.3125,NaN
312598,1980-12-16,AAPL,25.3125,NaN
312599,1980-12-17,AAPL,25.9375,NaN
312600,1980-12-18,AAPL,26.6875,NaN


In [254]:
import pandas as pd
df = df.dropna(subset=['rdq'])

# Merge the dataframes on 'tic' and ensure that the price is on or after 'rdq'
df = pd.merge_asof(df.sort_values('rdq'), 
                          df2.sort_values('date'), 
                          by='tic', 
                          left_on='rdq', 
                          right_on='date', 
                          direction='forward')

# Now merged_df will have an extra column with the stock price for the date closest to 'rdq'
df = df.sort_values(by=['tic', 'rdq'])
df.head()


,gvkey,datadate,tic,ajexq,rdq,actq,cshoq,epsfiq,teqq,cshfdy,oancfy,dvpspq,date,PRC,OPENPRC
1683,1690,1981-06-30,AAPL,224.0,1981-07-22,201.262,55.136,0.21,NaN,NaN,NaN,0.0,1981-07-22,22.7500,NaN
1726,1690,1981-09-30,AAPL,224.0,1981-10-20,227.104,55.309,0.19,NaN,NaN,NaN,0.0,1981-10-20,19.6875,NaN
1772,1690,1981-12-31,AAPL,224.0,1982-01-22,249.160,55.421,0.24,NaN,NaN,NaN,0.0,1982-01-22,20.8125,NaN
1812,1690,1982-03-31,AAPL,224.0,1982-04-15,263.140,56.488,0.24,NaN,NaN,NaN,0.0,1982-04-15,16.4375,NaN
1864,1690,1982-06-30,AAPL,224.0,1982-07-16,283.437,56.743,0.26,NaN,NaN,NaN,0.0,1982-07-16,13.3125,NaN


In [255]:
file_path_3 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/s&p500index.csv' # s&p500
df3 = pd.read_csv(file_path_3)
df3['caldt'] = pd.to_datetime(df3['caldt'])  # Adjust column name if needed

df3.tail()

,caldt,spindx
25794,2023-12-22,4754.63
25795,2023-12-26,4774.75
25796,2023-12-27,4781.58
25797,2023-12-28,4783.35
25798,2023-12-29,4769.83


In [256]:
df = df.sort_values(by=['rdq'])


#merge (fill empty cells with nearest s&p500 value)
df = pd.merge_asof(df, df3, left_on='rdq', right_on='caldt', direction='nearest')
df = df.drop(columns=['caldt'])
df = df[df['date'] <= '2023-12-29']

df['PRC'] = df['PRC'] / df['ajexq']
df['OPENPRC'] = df['OPENPRC'] / df['ajexq']
df['epsfiq'] = df['epsfiq'] / df['ajexq']
df['cshoq'] = df['cshoq'] * df['ajexq']
df['cshfdy'] = df['cshfdy'] * df['ajexq']

df = df.sort_values(by=['tic', 'rdq']).reset_index(drop=True)
df.tail()

,gvkey,datadate,tic,ajexq,rdq,actq,cshoq,epsfiq,teqq,cshfdy,oancfy,dvpspq,date,PRC,OPENPRC,spindx
15170,13721,2022-09-30,ZTS,1.0,2022-11-03,6551.0,466.442,1.13,4662.0,471.600,1171.0,0.325,2022-11-03,131.14000,134.14999,3719.89
15171,13721,2022-12-31,ZTS,1.0,2023-02-14,7506.0,463.808,0.99,4403.0,470.385,1912.0,0.325,2023-02-14,171.89999,173.44000,4136.13
15172,13721,2023-03-31,ZTS,1.0,2023-05-04,6269.0,462.495,1.19,4491.0,464.600,549.0,0.375,2023-05-04,178.47000,168.00000,4061.22
15173,13721,2023-06-30,ZTS,1.0,2023-08-08,6183.0,460.751,1.45,4621.0,463.800,732.0,0.375,2023-08-08,189.30000,182.14999,4499.38
15174,13721,2023-09-30,ZTS,1.0,2023-11-02,6222.0,459.525,1.29,5074.0,463.000,1456.0,0.375,2023-11-02,160.91000,155.31000,4317.78


In [257]:
# compute ratios
# Use OPENPRC first, if NaN then fall back to PRC

df['ptb'] = (df['OPENPRC'].fillna(df['PRC']) * df['cshoq']) / df['teqq']
df['div_yield'] = df['dvpspq'] / df['OPENPRC'].fillna(df['PRC'])
df['pe_inc'] = df['epsfiq'] / df['ajexq']
df['pcf'] = (df['OPENPRC'].fillna(df['PRC']) * df['cshfdy']) / df['oancfy']


df.tail(60)


,gvkey,datadate,tic,ajexq,rdq,actq,cshoq,epsfiq,teqq,cshfdy,oancfy,dvpspq,date,PRC,OPENPRC,spindx,ptb,div_yield,pe_inc,pcf
15115,4503,2020-06-30,XOM,1.0,2020-07-31,53016.0,4228.233,-0.26,187153.0,4270.000,6274.0,0.870,2020-07-31,42.08000,41.16000,3271.12,0.929903,0.021137,-0.26,28.012942
15116,4503,2020-09-30,XOM,1.0,2020-10-30,48693.0,4228.234,-0.15,184612.0,4270.000,10663.0,0.870,2020-10-30,32.62000,32.41000,3269.96,0.742298,0.026844,-0.15,12.978590
15117,4503,2020-12-31,XOM,1.0,2021-02-02,44893.0,4233.000,-4.70,164130.0,4271.000,14668.0,0.870,2021-02-02,45.63000,45.81000,3826.31,1.181464,0.018991,-4.70,13.338868
15118,4503,2021-03-31,XOM,1.0,2021-04-30,48195.0,4233.539,0.64,164101.0,4272.000,9264.0,0.870,2021-04-30,57.24000,58.33000,4181.17,1.504819,0.014915,0.64,26.898290
15119,4503,2021-06-30,XOM,1.0,2021-07-30,52842.0,4233.563,1.10,165556.0,4274.000,18914.0,0.870,2021-07-30,57.57000,58.88000,4395.26,1.505667,0.014776,1.10,13.305124
15120,4503,2021-09-30,XOM,1.0,2021-10-29,55555.0,4233.567,1.57,167506.0,4275.000,31005.0,0.870,2021-10-29,64.47000,65.05000,4605.38,1.644082,0.013374,1.57,8.969158
15121,4503,2021-12-31,XOM,1.0,2022-02-01,59154.0,4239.000,2.08,175683.0,4275.000,48129.0,0.880,2022-02-01,80.83000,76.45000,4546.54,1.844638,0.011511,2.08,6.790578
15122,4503,2022-03-31,XOM,1.0,2022-04-29,77255.0,4212.543,1.28,176526.0,4266.000,14788.0,0.880,2022-04-29,85.25000,88.14000,4131.93,2.103336,0.009984,1.28,25.426375
15123,4503,2022-06-30,XOM,1.0,2022-07-29,93163.0,4167.636,4.21,184508.0,4248.000,34751.0,0.880,2022-07-29,96.93000,94.45000,4130.29,2.133421,0.009317,4.21,11.545671
15124,4503,2022-09-30,XOM,1.0,2022-10-28,99289.0,4118.293,4.68,193043.0,4227.000,59176.0,0.880,2022-10-28,110.70000,109.46000,3901.06,2.335171,0.008039,4.68,7.818836


In [258]:


# convert features to pct

features = [
    'actq', 'ancq', 'epsfxq',
    'lctq', 'ltq', 'revtq',
    'capxy']

ratios = [
    'roa', 'roe','pe_inc', 'divyield',
    'pcf', 'roce',
    'debt_ebitda', 'gpm','evm',
    'cfm', 'npm', 'PEG_trailing',
    'de_ratio', 'curr_ratio', 'ptb']

df[features] = df[features].pct_change()

for col in ratios:
    df[col + '_change'] = df[col].pct_change()


df['prccq_change'] =  df['prccq'].pct_change()
df['next_prccq_change'] = df['prccq_change'].shift(-1)

df['spindx_change'] = df['spindx'].pct_change()
df['next_spindx_change'] = df['spindx_change'].shift(-1)

df['relative_quarterly_return'] = (df['prccq_change'] - df['spindx_change'])
df['next_relative_quarterly_return'] = df['relative_quarterly_return'].shift(-1)

#df = df.drop(columns=['prccq', 'spindx'])

df.info()

KeyError: "['ancq', 'epsfxq', 'lctq', 'ltq', 'revtq', 'capxy'] not in index"

In [ ]:
# REMOVE FALSE VALUES CAUSED BY PCT CHANGE

# Create a list to track the indices to drop
indices_to_drop = []

# Initialize the previous ticker as None
previous_ticker = None

# Iterate through the DataFrame, skipping the first row
for idx, row in df.iterrows():
    current_ticker = row['tic']
    
    # If the current ticker is different from the previous one, mark the current row for deletion
    if previous_ticker is not None and current_ticker != previous_ticker:
        indices_to_drop.append(idx)  # Append the index of the first occurrence of the new ticker
        indices_to_drop.append(idx-1)  # Append the index of the first occurrence of the new ticker
    
    # Update the previous ticker
    previous_ticker = current_ticker

# Drop the rows with the indices in the list
df = df.drop(indices_to_drop)

# Reset the index of the DataFrame after dropping rows
df = df.reset_index(drop=True)

df = df.drop(columns = ['index'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15339 entries, 0 to 15338
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   gvkey                           15339 non-null  int64         
 1   datadate                        15339 non-null  datetime64[ns]
 2   tic                             15339 non-null  object        
 3   actq                            15338 non-null  float64       
 4   ancq                            15338 non-null  float64       
 5   epsfxq                          15337 non-null  float64       
 6   lctq                            15338 non-null  float64       
 7   ltq                             15338 non-null  float64       
 8   revtq                           15338 non-null  float64       
 9   capxy                           14787 non-null  float64       
 10  prccq                           15338 non-null  float64       
 11  ro

In [ ]:

df = df.sort_values(by=['tic','datadate'])

df_temp = df[['spindx', 'spindx_change', 'next_spindx_change', 'prccq', 'prccq_change', 'next_prccq_change']]

df.head()


#df.to_csv('processed_data.csv', index=False)  # Set index=False to exclude the index column

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,prccq,roa,roe,pe_inc,divyield,pcf,roce,debt_ebitda,gpm,evm,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,spindx,roa_change,roe_change,pe_inc_change,divyield_change,pcf_change,roce_change,debt_ebitda_change,gpm_change,evm_change,cfm_change,npm_change,PEG_trailing_change,de_ratio_change,curr_ratio_change,ptb_change,prccq_change,next_prccq_change,spindx_change,next_spindx_change,relative_quarterly_return,next_relative_quarterly_return
0,1690,1980-12-31,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.124969,0.851,0.868,80.303,NaN,-74.325,1.240,0.027,0.433,13.535,0.112,0.100,NaN,0.556,2.612,11.083,135.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.282051,NaN,0.001768,NaN,-0.283819
1,1690,1981-03-31,AAPL,0.188251,0.072247,0.142857,0.365898,0.404152,0.164801,NaN,24.499985,0.606,0.468,77.035,NaN,-85.319,1.587,0.020,0.440,17.753,0.112,0.104,NaN,0.420,3.234,12.188,136.00,-0.287897,-0.460829,-0.040696,NaN,0.147918,0.279839,-0.259259,0.016166,0.311636,0.000000,0.040000,NaN,-0.244604,0.238132,0.099702,-0.282051,0.061225,0.001768,-0.035221,-0.283819,0.096445
2,1690,1981-06-30,AAPL,0.162343,0.198897,0.312500,0.265953,0.333577,0.151768,NaN,25.999985,0.539,0.388,34.698,NaN,-56.890,2.136,0.028,0.463,18.791,0.112,0.115,NaN,0.368,3.681,6.507,131.21,-0.110561,-0.170940,-0.549581,NaN,-0.333208,0.345936,0.400000,0.052273,0.058469,0.000000,0.105769,NaN,-0.123810,0.138219,-0.466114,0.061225,-0.413461,-0.035221,-0.114549,0.096445,-0.298912
3,1690,1981-09-30,AAPL,0.128400,0.315343,-0.095238,0.408022,0.329220,0.076709,NaN,15.249996,0.467,0.376,26.607,NaN,-44.727,0.583,0.182,0.487,11.468,0.143,0.118,NaN,0.437,3.231,5.659,116.18,-0.133581,-0.030928,-0.233183,NaN,-0.213799,-0.727060,5.500000,0.051836,-0.389708,0.276786,0.026087,NaN,0.187500,-0.122249,-0.130321,-0.413461,0.450819,-0.114549,0.054829,-0.298912,0.395990
4,1690,1981-12-31,AAPL,0.097119,0.298983,0.263158,0.194536,0.199391,0.367278,NaN,22.124985,0.409,0.286,22.812,NaN,-180.620,0.429,0.031,0.503,14.316,0.140,0.114,NaN,0.400,3.492,5.086,122.55,-0.124197,-0.239362,-0.142632,NaN,3.038277,-0.264151,-0.829670,0.032854,0.248343,-0.020979,-0.033898,NaN,-0.084668,0.080780,-0.101255,0.450819,-0.237288,0.054829,-0.086414,0.395990,-0.150875


In [ ]:
file_path_4 = '/Users/edocampione/Desktop/Meng Engineering Science/4YP/data/pro_data.csv' # cleaned data
df = pd.read_csv(file_path_4)
df['datadate'] = pd.to_datetime(df['datadate'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15339 entries, 0 to 15338
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   gvkey                           15339 non-null  int64         
 1   datadate                        15339 non-null  datetime64[ns]
 2   tic                             15339 non-null  object        
 3   actq                            15338 non-null  float64       
 4   ancq                            15338 non-null  float64       
 5   epsfxq                          15337 non-null  float64       
 6   lctq                            15338 non-null  float64       
 7   ltq                             15338 non-null  float64       
 8   revtq                           15338 non-null  float64       
 9   capxy                           14787 non-null  float64       
 10  roa                             15297 non-null  float64       
 11  ro

In [ ]:
df[['spindx', 'spindx_change', 'next_spindx_change', 'prccq', 'prccq_change', 'next_prccq_change']] = df_temp

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15339 entries, 0 to 15338
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   gvkey                           15339 non-null  int64         
 1   datadate                        15339 non-null  datetime64[ns]
 2   tic                             15339 non-null  object        
 3   actq                            15338 non-null  float64       
 4   ancq                            15338 non-null  float64       
 5   epsfxq                          15337 non-null  float64       
 6   lctq                            15338 non-null  float64       
 7   ltq                             15338 non-null  float64       
 8   revtq                           15338 non-null  float64       
 9   capxy                           14787 non-null  float64       
 10  roa                             15297 non-null  float64       
 11  ro

In [ ]:
# Replace inf and -inf with NaN
df.replace(['inf', '-inf', np.inf, -np.inf], np.nan, inplace=True)

df.head()

,gvkey,datadate,tic,actq,ancq,epsfxq,lctq,ltq,revtq,capxy,roa,roe,pe_inc,divyield,pcf,roce,debt_ebitda,gpm,evm,cfm,npm,PEG_trailing,de_ratio,curr_ratio,ptb,roa_change,roe_change,pe_inc_change,divyield_change,pcf_change,roce_change,debt_ebitda_change,gpm_change,evm_change,cfm_change,npm_change,PEG_trailing_change,de_ratio_change,curr_ratio_change,ptb_change,relative_quarterly_return,next_relative_quarterly_return,spindx,spindx_change,next_spindx_change,prccq,prccq_change,next_prccq_change
0,1690,1980-12-31,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.851,0.868,80.303,NaN,-74.325,1.240,0.027,0.433,13.535,0.112,0.100,NaN,0.556,2.612,11.083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.283819,135.76,NaN,0.001768,34.124969,NaN,-0.282051
1,1690,1981-03-31,AAPL,0.188251,0.072247,0.142857,0.365898,0.404152,0.164801,NaN,0.606,0.468,77.035,NaN,-85.319,1.587,0.020,0.440,17.753,0.112,0.104,NaN,0.420,3.234,12.188,-0.287897,-0.460829,-0.040696,NaN,0.147918,0.279839,-0.259259,0.016166,0.311636,0.000000,0.040000,NaN,-0.244604,0.238132,0.099702,-0.283819,0.096445,136.00,0.001768,-0.035221,24.499985,-0.282051,0.061225
2,1690,1981-06-30,AAPL,0.162343,0.198897,0.312500,0.265953,0.333577,0.151768,NaN,0.539,0.388,34.698,NaN,-56.890,2.136,0.028,0.463,18.791,0.112,0.115,NaN,0.368,3.681,6.507,-0.110561,-0.170940,-0.549581,NaN,-0.333208,0.345936,0.400000,0.052273,0.058469,0.000000,0.105769,NaN,-0.123810,0.138219,-0.466114,0.096445,-0.298912,131.21,-0.035221,-0.114549,25.999985,0.061225,-0.413461
3,1690,1981-09-30,AAPL,0.128400,0.315343,-0.095238,0.408022,0.329220,0.076709,NaN,0.467,0.376,26.607,NaN,-44.727,0.583,0.182,0.487,11.468,0.143,0.118,NaN,0.437,3.231,5.659,-0.133581,-0.030928,-0.233183,NaN,-0.213799,-0.727060,5.500000,0.051836,-0.389708,0.276786,0.026087,NaN,0.187500,-0.122249,-0.130321,-0.298912,0.395990,116.18,-0.114549,0.054829,15.249996,-0.413461,0.450819
4,1690,1981-12-31,AAPL,0.097119,0.298983,0.263158,0.194536,0.199391,0.367278,NaN,0.409,0.286,22.812,NaN,-180.620,0.429,0.031,0.503,14.316,0.140,0.114,NaN,0.400,3.492,5.086,-0.124197,-0.239362,-0.142632,NaN,3.038277,-0.264151,-0.829670,0.032854,0.248343,-0.020979,-0.033898,NaN,-0.084668,0.080780,-0.101255,0.395990,-0.150875,122.55,0.054829,-0.086414,22.124985,0.450819,-0.237288


In [ ]:
df.to_csv('dati_trattati.csv', index=False)  # Set index=False to exclude the index column